# 导入库以及数据集

In [2]:
# 基本工具
import numpy as np
import pandas as pd 
import time
import os #修改环境变量

#算法/损失/评估指标等
import sklearn
from sklearn.ensemble import RandomForestRegressor as RFR # 随机森林回归
from sklearn.model_selection import KFold,cross_validate

# 优化器
from bayes_opt import BayesianOptimization

In [5]:
data = pd.read_csv(r"C:\Users\ZARD\Desktop\超参\程序学习\AutoML与超参数优化——菜菜\train_encode.csv",index_col=0)

# 查看数据集的信息

In [6]:
data.head(5)

,Id,住宅类型,住宅区域,街道接触面积(英尺),住宅面积,街道路面状况,巷子路面状况,住宅形状(大概),住宅现状,水电气,...,泳池面积,泳池质量,篱笆质量,其他配置,其他配置的价值,销售月份,销售年份,销售类型,销售状态,SalePrice
0,0.0,5.0,3.0,36.0,327.0,1.0,0.0,3.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2.0,8.0,4.0,208500
1,1.0,0.0,3.0,51.0,498.0,1.0,0.0,3.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,1.0,8.0,4.0,181500
2,2.0,5.0,3.0,39.0,702.0,1.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,8.0,2.0,8.0,4.0,223500
3,3.0,6.0,3.0,31.0,489.0,1.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,8.0,0.0,140000
4,4.0,5.0,3.0,55.0,925.0,1.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,11.0,2.0,8.0,4.0,250000


In [4]:
x = data.iloc[:,:-1]
y = data.iloc[:,-1]

In [5]:
x.head()

,Id,住宅类型,住宅区域,街道接触面积(英尺),住宅面积,街道路面状况,巷子路面状况,住宅形状(大概),住宅现状,水电气,...,半开放式门廊面积,泳池面积,泳池质量,篱笆质量,其他配置,其他配置的价值,销售月份,销售年份,销售类型,销售状态
0,0.0,5.0,3.0,36.0,327.0,1.0,0.0,3.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,8.0,4.0
1,1.0,0.0,3.0,51.0,498.0,1.0,0.0,3.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,8.0,4.0
2,2.0,5.0,3.0,39.0,702.0,1.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,8.0,2.0,8.0,4.0
3,3.0,6.0,3.0,31.0,489.0,1.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,8.0,0.0
4,4.0,5.0,3.0,55.0,925.0,1.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,11.0,2.0,8.0,4.0


In [7]:
y.head() # 价格

0    208500
1    181500
2    223500
3    140000
4    250000
Name: SalePrice, dtype: int64

In [6]:
x.shape

(1460, 80)

# 做贝叶斯优化（BayesianOptimization）

## 定义目标函数

In [8]:
# 定义目标函数
def bayesopt_objective(n_estimators,max_depth,max_features,min_impurity_decrease):
    # 定义评估器
    # 需要调整的超参数等于目标函数的输入，不需要调整的超参数则直接等于固定值
    # 默认参数输入一定是浮点数，因此需要套上int函数处理成整数
    reg = RFR(
        n_estimators=int(n_estimators)
        ,max_depth=int(max_depth)
        ,max_features=int(max_features)
        ,min_impurity_decrease=min_impurity_decrease
        ,random_state=1412
        ,verbose=False # 可自行决定是否开启森林建树的verbose
        ,n_jobs=-1
    )
    
    # 定义损失的输出，5折交叉验证下的结果，输出负根均方误差（-RMSE）
    # 注意，交叉验证需要使用数据，但我们不能让数据x,y成为目标函数的输入
    cv = KFold(n_splits=5,shuffle=True,random_state=1412)
    validation_loss = cross_validate(
        reg,x,y
        ,scoring="neg_root_mean_squared_error" # 负根均方误差
        ,cv=cv
        ,verbose=False # 不需要打印具体流程
        ,n_jobs=-1 
        ,error_score='raise' # 默认值为None，此时返回“空”
        # 如果交叉验证中算法执行报错，则告诉我们错误的理由
    )
    # 交叉验证输出的评估指标是负根均方误差，因此本来就是负的损失
    # 目标函数可直接输出该损失的均值
    return np.mean(validation_loss["test_score"])

## 设置备选参数空间

In [9]:
#备选参数空间
param_grid_simple = {
    'n_estimators':(80,100)
    ,'max_depth':(10,25)
    ,'max_features':(10,20)
    ,'min_impurity_decrease':(0,1)
    }

## 定义优化目标函数的具体流程

In [9]:
# 定义优化目标函数的具体流程
def param_bayes_opt(init_points,n_iter):
    
    # 定义优化器，先实例化优化器
    opt = BayesianOptimization(
          bayesopt_objective # 需要优化的目标函数
        ,param_grid_simple # 备选参数空间
        ,random_state = 1412 # 随机数种子，虽然无法控制
    )
    
    # 使用优化器，其中bayes_opt只支持最大化
    opt.maximize(
        init_points = init_points  # 抽取多少个观测值
        ,n_iter = n_iter # 一共观测/迭代多少次
    )

    #优化完成，取出最佳最佳参数与分数
    params_best = opt.max['params']
    score_best = opt.max["target"]

    #打印最佳参数与最佳分数
    print("\n","\n","best params: ",params_best,
         "\n","\n","best cvscore: ",score_best_best,
         )
    
    # 返回最佳参数与最佳分数
    return params_best,score_best


## 定义验证函数

In [11]:
# 定义验证函数

def bayes_opt_validation(params_best):
    
    reg = RFR(
        n_estimators = int(params_best["n_estimators"])
        ,max_depth=int(params_best["max_depth"])
        ,min_impurity_decrease = params_best["min_impurity_decrease"]
        ,random_state=1412
        ,verbose=Flase
        ,n_jobs=-1
    )
    cv = KFold(n_splits=5,shuffle=True,random_state=1412)
    validation_loss = cross_validate(
        reg,x,y
        ,scoring = "neg_root_mean_squared_error"
        ,cv = cv
        ,verbose = False
        ,n_jobs = -1
    )# 执行实际优化流程
start = time.time()
params_best,score_best = param_bayes_opt(20,280) # 初始看20个观测值，后面迭代280次
print('It take %s minutes'% ((time.time()-start)/60))
validation_score = bayes_opt_validation(params_best)
print("\n","\n","validation_score: ",validation_score)
    
    return np.mean(validation_loss["test_score"])
    

## 执行实际优化流程

In [12]:
# 执行实际优化流程
start = time.time()
params_best,score_best = param_bayes_opt(20,280) # 初始看20个观测值，后面迭代280次
print('It take %s minutes'% ((time.time()-start)/60))
validation_score = bayes_opt_validation(params_best)
print("\n","\n","validation_score: ",validation_score)

|   iter    |  target   | max_depth | max_fe... | min_im... | n_esti... |
-------------------------------------------------------------------------
| 1         | -2.948e+0 | 23.2      | 17.52     | 0.06379   | 88.79     |
| 2         | -2.909e+0 | 14.8      | 17.61     | 0.9214    | 97.58     |
| 3         | -2.9e+04  | 15.86     | 15.56     | 0.2661    | 87.98     |
| 4         | -2.887e+0 | 14.05     | 16.84     | 0.06744   | 89.72     |
| 5         | -2.887e+0 | 18.71     | 19.17     | 0.9315    | 83.7      |
| 6         | -2.895e+0 | 17.7      | 19.58     | 0.7127    | 89.18     |
| 7         | -2.968e+0 | 14.21     | 12.62     | 0.3381    | 91.51     |
| 8         | -2.91e+04 | 23.23     | 10.89     | 0.6078    | 95.06     |
| 9         | -2.891e+0 | 14.89     | 14.0      | 0.9487    | 80.16     |
| 10        | -2.958e+0 | 11.52     | 12.58     | 0.03276   | 92.56     |
| 11        | -2.91e+04 | 13.14     | 13.31     | 0.2563    | 98.24     |
| 12        | -2.95e+04 | 17.94     | 

| 67        | -2.88e+04 | 19.42     | 17.58     | 0.0       | 92.78     |
| 68        | -2.932e+0 | 18.38     | 17.41     | 0.4656    | 92.52     |
| 69        | -2.895e+0 | 20.4      | 17.11     | 0.0       | 92.6      |
| 70        | -2.932e+0 | 19.99     | 18.57     | 0.0       | 92.48     |
| 71        | -2.92e+04 | 19.6      | 16.9      | 0.0       | 93.7      |
| 72        | -2.891e+0 | 17.3      | 19.87     | 0.9839    | 80.93     |
| 73        | -2.898e+0 | 20.3      | 17.89     | 0.9251    | 89.54     |
| 74        | -2.928e+0 | 19.45     | 18.65     | 0.03773   | 89.87     |
| 75        | -2.935e+0 | 18.64     | 17.52     | 0.9808    | 90.15     |
| 76        | -2.905e+0 | 17.7      | 20.0      | 1.0       | 85.45     |
| 77        | -2.894e+0 | 16.19     | 19.43     | 0.3418    | 80.88     |
| 78        | -2.885e+0 | 19.44     | 17.3      | 0.0       | 90.86     |
| 79        | -2.919e+0 | 16.83     | 18.84     | 1.0       | 85.54     |
| 80        | -2.886e+0 | 15.56     | 

| 134       | -2.913e+0 | 10.01     | 16.65     | 0.3608    | 89.52     |
| 135       | -2.855e+0 | 15.43     | 18.05     | 0.2011    | 83.39     |
| 136       | -2.9e+04  | 20.93     | 19.89     | 0.0004996 | 91.1      |
| 137       | -2.855e+0 | 15.39     | 18.38     | 0.873     | 83.47     |
| 138       | -2.907e+0 | 14.87     | 18.23     | 0.4442    | 84.1      |
| 139       | -2.878e+0 | 15.09     | 17.6      | 0.8051    | 83.35     |
| 140       | -2.852e+0 | 15.78     | 18.09     | 0.9062    | 81.62     |
| 141       | -2.908e+0 | 14.83     | 18.6      | 0.957     | 83.29     |
| 142       | -2.878e+0 | 15.56     | 17.85     | 0.9959    | 83.26     |
| 143       | -2.917e+0 | 16.17     | 17.32     | 0.2172    | 83.37     |
| 144       | -2.855e+0 | 15.84     | 18.57     | 0.9748    | 83.43     |
| 145       | -2.854e+0 | 15.63     | 18.62     | 0.8737    | 82.85     |
| 146       | -2.898e+0 | 15.65     | 19.02     | 0.7       | 83.23     |
| 147       | -2.891e+0 | 17.25     | 

| 202       | -2.865e+0 | 23.9      | 19.33     | 0.6924    | 90.25     |
| 203       | -2.888e+0 | 24.41     | 19.55     | 0.9851    | 91.42     |
| 204       | -2.89e+04 | 24.27     | 19.76     | 0.2488    | 90.17     |
| 205       | -2.865e+0 | 23.16     | 19.4      | 0.616     | 89.93     |
| 206       | -2.917e+0 | 23.05     | 18.71     | 0.3733    | 90.52     |
| 207       | -2.865e+0 | 23.52     | 19.64     | 0.3483    | 89.4      |
| 208       | -2.863e+0 | 23.53     | 19.91     | 0.8679    | 90.58     |
| 209       | -2.863e+0 | 23.12     | 19.86     | 0.8781    | 89.0      |
| 210       | -2.865e+0 | 23.53     | 19.9      | 0.8121    | 89.67     |
| 211       | -2.891e+0 | 22.89     | 19.86     | 0.8016    | 90.31     |
| 212       | -2.863e+0 | 23.57     | 19.2      | 1.0       | 89.3      |
| 213       | -2.862e+0 | 23.27     | 19.54     | 0.2629    | 88.52     |
| 214       | -2.862e+0 | 23.88     | 19.79     | 0.5593    | 88.66     |
| 215       | -2.867e+0 | 23.76     | 

| 270       | -2.949e+0 | 25.0      | 17.7      | 1.0       | 80.0      |
| 271       | -2.907e+0 | 25.0      | 15.46     | 1.0       | 80.0      |
| 272       | -2.871e+0 | 23.44     | 19.99     | 0.6317    | 84.55     |
| 273       | -2.904e+0 | 22.99     | 19.29     | 0.6946    | 84.0      |
| 274       | -2.928e+0 | 25.0      | 10.0      | 0.0       | 100.0     |
| 275       | -2.887e+0 | 24.14     | 19.84     | 0.004388  | 97.23     |
| 276       | -2.899e+0 | 24.36     | 19.94     | 0.8132    | 83.4      |
| 277       | -2.864e+0 | 23.24     | 19.96     | 0.9658    | 97.67     |
| 278       | -2.895e+0 | 10.0      | 20.0      | 0.0       | 94.02     |
| 279       | -2.868e+0 | 23.42     | 19.82     | 0.02389   | 94.61     |
| 280       | -2.937e+0 | 18.25     | 10.0      | 1.0       | 94.42     |
| 281       | -2.95e+04 | 10.09     | 11.56     | 0.2629    | 80.03     |
| 282       | -2.905e+0 | 10.0      | 20.0      | 1.0       | 85.86     |
| 283       | -2.891e+0 | 18.31     | 

NameError: name 'score_best_best' is not defined